# Training Spacey
### Timbuktu Chronicles
Some linkes regarding the training data:
- https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718
- https://www.machinelearningplus.com/training-custom-ner-model-in-spacy/ 
- https://www.youtube.com/watch?v=YBRF7tq1V-Q 
  - and the corresponding code file: https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_02_customizing_spacy.py

To-do:
- See how to get the output of the spacey NER into a file so you can edit it 
(might have to do with the save_data function in the code for the video above - and also for the code in the lesson before it, 4.1)
- Correct the spacey output
- Use the corrected output to upload back to Spacey and use for training

Refer to TimbuktuChronicles Google Colab notebook for some initial examples with Spacy and file access.

Some segments of my code was based off of the YouTube video https://www.youtube.com/watch?v=YBRF7tq1V-Q , and corresponding code file: https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_02_customizing_spacy.py


In [ ]:
import spacy
from spacy import displacy
from collections import Counter
#import en_core_web_sm
#import en_core_web_md
#import en_core_web_lg
#import en_core_web_trf
from pprint import pprint
import json
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
def test_model(model, text):
    doc = model(text)
    results = []
    entities = []
    for ent in doc.ents:
        entities.append((ent.start_char, ent.end_char, ent.label_))
    if len(entities) > 0:
        results = [text, {"entities": entities}]
        return (results)

In [ ]:
def save_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [ ]:
# I did this once so I don't need to do it again
!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_lg 
#!python -m spacy download en_core_web_trf

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp37-none-any.whl size=98051304 sha256=89e9fad14984e9397384c8ad9abb96c63afeda5d12f3853b42f34e8a9b2ad030
  Stored in directory: /tmp/pip-ephem-wheel-cache-njvxy_6x/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import en_core_web_md

In [ ]:
import re

In [ ]:
# You don't really need to run this, just do it for whenever you're adding a new sample so you have the initial JSON file to modify/correct.
#splitting the text by punctuation instead of lines 
#file1 = open("/content/gdrive/My Drive/fattashEnglishSample.txt", "r")
file2 = open("/content/gdrive/My Drive/soudanEnglishSample.txt", "r")
#text = re.split('[?.!;:]', file1.read()) + re.split('[?.!;:]', file2.read()) 
text = re.split('[?.!;:]', file2.read())

for i in range(len(text)):
  text[i] = text[i].replace('\n', ' ')

######################


INITIAL_DATA = []
#nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()
for line in text:
  line = line.strip()
  results = test_model(nlp, line)
  if results != None:
    INITIAL_DATA.append(results)

'''
INITIAL_DATA = []
file1 = open("/content/gdrive/My Drive/fattashEnglishSample.txt", "r")
file2 = open("/content/gdrive/My Drive/soudanEnglishSample.txt", "r")
nlp = spacy.load("en_core_web_md") #en_core_web_sm.load()
for line in file1.readlines():
  line = line.strip()
  results = test_model(nlp, line)
  if results != None:
     INITIAL_DATA.append(results)

for line in file2.readlines():
  line = line.strip()
  results = test_model(nlp, line)
  if results != None:
    INITIAL_DATA.append(results)
'''
#save_data("/content/gdrive/My Drive/fattash_initial_output.json", INITIAL_DATA)
#save_data("/content/gdrive/My Drive/soudan_initial_output.json", INITIAL_DATA)
len(INITIAL_DATA)
save_data("/content/gdrive/My Drive/soudanfattash_punctuationseparated_initial_output.json", INITIAL_DATA)

'\nINITIAL_DATA = []\nfile1 = open("/content/gdrive/My Drive/fattashEnglishSample.txt", "r")\nfile2 = open("/content/gdrive/My Drive/soudanEnglishSample.txt", "r")\nnlp = spacy.load("en_core_web_md") #en_core_web_sm.load()\nfor line in file1.readlines():\n  line = line.strip()\n  results = test_model(nlp, line)\n  if results != None:\n     INITIAL_DATA.append(results)\n\nfor line in file2.readlines():\n  line = line.strip()\n  results = test_model(nlp, line)\n  if results != None:\n    INITIAL_DATA.append(results)\n'

Code for training the model is based off of the following 3 links:
  - https://www.machinelearningplus.com/training-custom-ner-model-in-spacy/
  - https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7
  - https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_03_customizing_spacy.py


In [ ]:
# load the training data back from the JSON file, via https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_03_customizing_spacy.py
def load_data(files):
  data = []
  for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data += json.load(f)
  return (data)


In [ ]:
#TRAIN_DATA = load_data(["/content/gdrive/My Drive/fattash_training_data.json", "/content/gdrive/My Drive/soudan_training_data.json"])
TRAIN_DATA = load_data(["/content/gdrive/My Drive/soudanfattash_punctuationseparated_training.json"])
#TRAIN_DATA = load_data(["/content/gdrive/My Drive/soudanfattash_punctuationseparated_trainingTWO.json"])


In [ ]:
len(TRAIN_DATA)

248

In [ ]:
# split the data into TRAINING and TESTING
random.shuffle(TRAIN_DATA)
TRAIN = TRAIN_DATA[:int(.8*len(TRAIN_DATA))]
TEST = TRAIN_DATA[int(.8*len(TRAIN_DATA)):]

In [ ]:
print(len(TRAIN))
print(len(TEST))

198
50


In [ ]:
nlp = en_core_web_md.load()

In [ ]:
# NLP prediction PRE-TRAINING - this predicts the entities/labels for the TEXT - it completely ignores the entities and predicts its own
#nlp = spacy.load('en_core_web_md')

predicted_labels = []
predicted_labels2 = []
predicted_labels3 = []
for text, _ in TRAIN:
    doc = nlp(text)
    predicted_labels.append([ent.label_ for ent in doc.ents])
    predicted_labels2.append([ent for ent in doc.ents])
    predicted_labels3.append([(ent.text, ent.label_) for ent in doc.ents])


In [ ]:
# these are the entities as manually labelled by me, above are predicted by initial NLP model
correct_labels = []
correct_labels3 = []
for _text, annotations in TRAIN:
  #print(_text)
 # print(annotations.get('entities'))
  correct_labels.append([ent[2] for ent in annotations.get('entities')])
  correct_labels3.append([(_text[ent[0]:ent[1]], ent[2]) for ent in annotations.get('entities')])

#do I have to use the entities' offsets to index _ to get the actual text? *sigh*

In [ ]:
# just testing for the accuracy code, don't need to run it
for i in range(len(correct_labels3)):
  print(correct_labels3[i])
  print(predicted_labels3[i])

  for j in range(len(correct_labels3[i])):
    print(correct_labels3[i][j] in predicted_labels3[i])

#  print(('Mohammed', 'PERSON') in correct_labels3[i])
  #print(('Mohammed', 'PERSON') == ('Mohammed', 'PERSON'))
  #print([value for value in correct_labels3[i] if value in predicted_labels3[i]])
  #for j in range(len(correct_labels3[i])):
   # print(correct_labels3[i][j])
    #print(len(correct_labels3[i]) == len(predicted_labels3[i]))
    #print([value for value in ])

[("Shi'i", 'NORP'), ('God', 'TITLE')]
[]
False
False
[('Askia-El-Hâdj-Mohammed', 'PERSON'), ("'Ali-Folen", 'PERSON')]
[('Askia-El-Hâdj-Mohammed', 'PERSON'), ('Ali-Folen', 'PERSON')]
True
False
[('Ahmed Bibokor', 'PERSON')]
[('Ahmed Bibokor', 'PERSON')]
True
[('cadi', 'TITLE'), ('Mahmud', 'PERSON'), ('imam', 'TITLE'), ('Sankore', 'GPE')]
[('cadi', 'PERSON'), ('Mahmud', 'PERSON'), ('Sankore', 'PERSON')]
False
True
False
False
[('God', 'TITLE')]
[]
False
[('That year', 'DATE'), ('cadi', 'TITLE'), ('Habib', 'PERSON'), ('God', 'TITLE')]
[('That year', 'DATE'), ('cadi Habib', 'PERSON')]
True
False
False
False
[('Kàgho', 'GPE'), ('Kabara', 'GPE'), ('Mahmud', 'PERSON'), ('Kàgho', 'GPE')]
[('Kàgho', 'PERSON'), ('Kabara', 'GPE'), ('Mahmud', 'PERSON'), ('Kàgho', 'PERSON')]
False
True
True
False
[('one half', 'CARDINAL'), ('Kalo', 'PERSON'), ('Salta-Tayenda', 'PERSON'), ('half', 'CARDINAL'), ('Domel', 'PERSON'), ('caid', 'TITLE'), ('the Sultan of Djolf', 'PERSON')]
[('one half', 'CARDINAL'), ('Kal

In [ ]:
# FUNCTION takes two lists of entities lists (one as the predicted and one as the corrected) as input and outputs the accuracy
# this was for the format of the predicted_labels and correct_labels array
'''
def test_accuracy(predicted_labels, correct_labels):
  correct_predictions = 0.0
  total_entities = 0.0
  for i in range(len(correct_labels)): # or predicted_labels; it prob doesn't matter since they're prob both the same size
  # checking entity by entitiy:

    if len(correct_labels[i]) >= len(predicted_labels[i]):
      for j in range(len(correct_labels[i])):
        total_entities += 1
        try:
          if correct_labels[i][j] == predicted_labels[i][j]:
            correct_predictions += 1
        except: # if j is out of range for predicted_labels[i]
            break
    else: # if len(predicted_labels[i]) > correct_labels[i]
      for j in range(len(predicted_labels[i])):
        total_entities += 1
        try:
          if predicted_labels[i][j] == correct_labels[i][j]:
            correct_predictions += 1
        except: # if j is out of range for predicted_labels[i]
          break


  return (correct_predictions / total_entities) 
'''

def test_accuracy(predictedLabels, correctLabels):
  correct_predictions = 0.0
  total_entities = 0.0  
  assert len(correctLabels) == len(predictedLabels)

  for i in range(len(correctLabels)):
    for j in range(len(correctLabels[i])):
      total_entities += 1 # count the total entities based on the number in the correctLabels array
      if correctLabels[i][j] in predictedLabels[i]:
        correct_predictions += 1

  return (correct_predictions / total_entities)




In [ ]:
#print(test_accuracy(predicted_labels, correct_labels))
print(test_accuracy(predicted_labels3, correct_labels3))

0.49907235621521334


In [ ]:
# TRAINING THE MODEL 

# via https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

# load a spacy model
#nlp = spacy.blank('en') - nvrmind ABOVE I already set nlp = en_core_web_sm.load() \

# make sure ner is in the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

# add the labels from TRAIN_DATA into the ner:
for _, annotations in TRAIN:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# maybe here is where I would split the TRAINING/TESTING DATA

#we only want to train NER, so this code tells it to ignore training everything else
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(30): # customize the number of iterations
        #random.shuffle(TRAIN)
        shuffledTRAIN = random.sample(TRAIN, len(TRAIN))
        losses = {}
        for text, annotations in tqdm(shuffledTRAIN):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses) # not sure why I have this 100% - 3% pattern or why the numbers following 'ner' are so high. Maybe I should start with a different spacy model for nlp?

  2%|▏         | 3/198 [00:00<00:08, 22.15it/s]

{'ner': 4428.99342587261}


  1%|          | 2/198 [00:00<00:09, 19.79it/s]

{'ner': 3948.8919323061227}


  2%|▏         | 3/198 [00:00<00:08, 22.29it/s]

...

{'ner': 3461.364826050963}


  1%|          | 2/198 [00:00<00:14, 13.34it/s]

{'ner': 3482.345795247266}


100%|██████████| 198/198 [00:13<00:00, 14.37it/s]

{'ner': 3482.1395444139716}


In [ ]:
# NLP prediction POST-TRAINING - this predicts the entities/labels for the TEXT - it completely ignores the entities and predicts its own
predicted_labels = []
for text, _ in TEST:
    doc = nlp(text)
    predicted_labels.append([ent.label_ for ent in doc.ents])

In [ ]:
len(predicted_labels)

50

In [ ]:
# these are the entities as manually labelled by me, above are predicted by TRAINED NLP model
correct_labels = []
for _, annotations in TEST:
    correct_labels.append([ent[2] for ent in annotations.get('entities')])

In [ ]:
len(correct_labels)

50

In [ ]:
print(test_accuracy(predicted_labels, correct_labels))

0.8095238095238095


In [ ]:
# saving/reloading the model so you don't have to train each time, via https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

# saving the model:
"""
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
"""

# reloading the model
# model = spacy.load('model_name')

# IMPORTANT:SEE "generate_rules" and "nlp = spacy.load("hp_ner")" in https://github.com/wjbmattingly/ner_youtube/blob/main/lessons/04_01_customizing_spacy.py. 
# It seems like you need to save the model that you train to re-use it.


In [ ]:
# visuals
# I think you'd have to do something like the following
'''
data = load_data(["/content/gdrive/My Drive/fattash_training_data.json", "/content/gdrive/My Drive/soudan_training_data.json"])
for text, _ in data:
    doc = nlp(text)
    and then displacy.render, etc.
'''
#displacy.render(doc, jupyter=True, style='ent') #- coloring the entities
# displacy.render(doc, style='dep', jupyter = True, options = {'distance': 120}) - graph

In [ ]:
#data = load_data(["/content/gdrive/My Drive/fattash_training_data.json", "/content/gdrive/My Drive/soudan_training_data.json"])
data = load_data(["/content/gdrive/My Drive/soudanfattash_punctuationseparated_training.json"])
for text, _ in data:
    doc = nlp(text)
    displacy.render(doc, jupyter=True, style='ent')

/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


...

/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)
